# CS 5683 - Big Data Analytics
## Assignment - 1: Intro. to Spark and RDD

###### Use Google Colab to use this notebook
###### Let's setup Spark first

In [ ]:
!pip install pyspark

     |████████████████████████████████| 204.2MB 69kB/s 
     |████████████████████████████████| 204kB 53.8MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612243 sha256=d6719781f128b1a24589ca17b0276e6fb5008652165f630fd028443f5b6d89e0
  Stored in directory: /root/.cache/pip/wheels/5e/bd/07/031766ca628adec8435bb40f0bd83bb676ce65ff4007f8e73f
Successfully built pyspark


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


###### Import required libraries now

In [ ]:
import os
import sys
from pyspark import SparkContext, SparkConf

###### Let's initialize Spark context now

In [ ]:
# create Spark context with necessary configuration
sc = SparkContext("local","PySpark Word Count Exmaple")

###### Follow the tutorial to mount your Google Drive. Give mounted Drive paths below

In [ ]:
# Give **.txt FILE PATHS** here
file1 = '/content/drive/My Drive/file1.txt' # change the path  
file2 = '/content/drive/My Drive/file2.txt' # change the path  

filer = open('/content/drive/My Drive/file1.txt', 'r') 
Lines = filer.readlines()

count = 0
# Strips the newline character 
for line in Lines: 
    print("Line{}: {}".format(count, line.strip())) 


# USE THESE FILES as input(s) FOR ALL BELOW QUESTIONS

Line0: my name is Suraj
Line0: my name is SuraJ
Line0: my name is Suraj
Line0: my name is Suraj1


### Example Spark program

In [ ]:
fileRDD = sc.textFile(file1)
wordsRDD = fileRDD.flatMap(lambda line: line.strip().split(' '))
wordsRDD = wordsRDD.filter(lambda word: word.lower()[-1].isdigit() == True)
# pairRDD = wordsRDD.map(lambda word: (word,1))
# countRDD = pairRDD.reduceByKey(lambda a,b: a+b)
wordsRDD.collect()

['Suraj1']

In [ ]:
# Example Spark application for a simple wordcount
# What is wordcount? 
    # Given a file, count the frequency of all words appearing in that file
    
# Step-1: Read the required file. In our case it is file1 or file2.
# NOTE: We do not need to initialize SparkContext as only one SparkContext can be initialized in one notebook
fileRDD = sc.textFile(file1)

# Step-2: 
    # Each line in our file(s) is a sentence. So, we need to split the sentence with ' ' to get words
    # Using map() will return RDD[list]. But we need RDD[string]. So we use flatMap()
wordsRDD = fileRDD.flatMap(lambda line: line.split(" ")) # <----------- TEST what happens when you use map()

wordsRDDm = fileRDD.map(lambda line: line.split(" ")) # <----------- TEST what happens when you use map()

# Step-3: For each input, we will make (K,V) pair, where K is the word and V is 1
pairRDD = wordsRDDm.map(lambda word: (word,1))

# Step-4: Now we have to sum all 1's of each word
# NOTE: A word may present in multiple data partitions. So we use reduceByKey() to group by key and perform sum
countRDD = pairRDD.reduceByKey(lambda a,b: a+b)

#Step-5: Save results in a text file
#pairRDD.saveAsTextFile('/content/drive/My Drive/Coursework/CS 5683_Big_Data/file13.txt') # <----------- GIVE FILE PATH
countRDD.saveAsTextFile('/content/drive/My Drive/file10.txt') # <----------- GIVE FILE PATH

### Question - 1 (10 points) 

Write a Spark application that counts number of words that ends with each letter in file1. That is for each letter count the number of (non-unique) words that ends with the specific letter. You can **ignore** letter cases (consider the given text contains only lower-case letters). Also, you can **ignore** words that end with non-alphabetic letters. **Sort the output in alphabetical order**

> Indented block



Example Output:
('a', 500)
('B', 100)

which means that the given input has 500 words that end with the letter 'a' and 100 words that end with letter 'B'.
NOTE: The output is sorted, the application counts duplicate words also, and the application is not case-sensitive

In [ ]:
# YOUR CODE for Question-1 HERE

In [ ]:
fileRDD = sc.textFile(file1)

wordsRDD = fileRDD.flatMap(lambda line: line.split(" ")) # <-------------------- TEST what happens when you use map()

wordsRDDf = wordsRDD.filter(lambda word: word.lower()[-1].isdigit() == False) # remove words that ends with digits

pairRDD = wordsRDDf.map(lambda word: (word[-1],1)) # map with key = last letter of word, and value = 1

countRDD = pairRDD.reduceByKey(lambda a,b: a+b) # reduce keys with summation operation

countRDDsort = countRDD.sortByKey(ascending=True) # sort using keys in ascending order

countRDDsort.saveAsTextFile('/content/drive/My Drive/file_sol_q1.txt') # <----------- GIVE FILE PATH


In [ ]:
# PRINT THE OUTPUT HERE
for x in countRDDsort.collect():
  print(x)

('J', 1)
('e', 4)
('j', 2)
('s', 4)
('y', 4)


### Question - 2 (10 points)

Write a Spark application that counts the **unique** number of words that have **n-letters** in file1. That is count the number of **unique** 1-letter words, 2-letter words, 3-letter words, etc. You can **include** all non-alphabetic letters for this application.

Example Output:
(1, 100)
(2, 700)
(3, 1500)

which means that the input file for the Spark application has **100 unique 1-letter words** , **700 unique 2-letter words** , and **1500 uniqe 3-letter words**

In [ ]:
# YOUR CODE for Question-2 HERE

In [ ]:
fileRDD = sc.textFile(file1)

wordsRDD = fileRDD.flatMap(lambda line: line.split(" ")) # <----------- TEST what happens when you use map()

distinctwordsRDD = wordsRDD.distinct() # remove duplicate entries

pairRDD = distinctwordsRDD.map(lambda word: (len(word),1)) # map with key = length of word, and value = 1

countRDD = pairRDD.reduceByKey(lambda a,b: a+b) # reduce keys with summation operation

countRDDsort = countRDD.sortByKey(ascending=True) # sort using keys in ascending order

countRDDsort.saveAsTextFile('/content/drive/My Drive/file_sol_q2.txt') # <----------- GIVE FILE PATH

In [ ]:
# PRINT THE OUTPUT HERE
for x in countRDDsort.collect():
  print(x)

### Question - 3 (10 points)

Write a Spark aplication that outputs wordcount from two files: file1 and file2. That is count the number of occurances of words from two files. You can **ignore** letter cases (consider the given input files contain only lower-case letters). Also, you can **ignore** that is not present in both files.

Example Output:
(big, (10, 20))
(Data, (30, 50))

which means the word "big" appears 10 times in file-1 and 20 times in file-2 and the word "Data" appears 30 times in file-1 and 50 times in file-2

In [ ]:
# YOUR CODE for Question-3 HERE

In [ ]:
fileRDD1 = sc.textFile(file1)
fileRDD2 = sc.textFile(file2)

wordsRDD1 = fileRDD1.flatMap(lambda line: line.split(" ")) # <----------- TEST what happens when you use map()
wordsRDD2 = fileRDD2.flatMap(lambda line: line.split(" ")) # <----------- TEST what happens when you use map()

pairRDD1 = wordsRDD1.map(lambda word: (word,1)) # map with key = word, and value = 1
pairRDD2 = wordsRDD2.map(lambda word: (word,1)) # map with key = word, and value = 1

countRDD1 = pairRDD1.reduceByKey(lambda a,b: a+b) # reduce keys with summation operation
countRDD2 = pairRDD2.reduceByKey(lambda a,b: a+b) # reduce keys with summation operation

countRDD3 = countRDD1.join(countRDD2).reduceByKey(lambda x,y : (x,y)) # join operation
#countRDD3 = countRDD1.union(countRDD2).reduceByKey(lambda x,y : (x,y)) # union operation

# countRDD3.saveAsTextFile('/content/drive/My Drive/file_sol_q3.txt') # <----------- GIVE FILE PATH

In [ ]:
# PRINT THE OUTPUT HERE
for x in countRDD3.collect():
  print(x)

('my', (4, 2))
('name', (4, 2))
('is', (4, 2))
('Suraj', (2, 1))


### WHAT TO TURN-IN IN CANVAS

1. Complete questions 1,2, and 3
2. Download this completed python notebook as .ipynb
3. Upload it in Canvas assignment

# Due Date: Sept. 3 at 11:59pm

In [ ]:
rdd = sc.parallelize(data)
rdd = rdd.filter(lambda x: x[1] == 'Biology').map(lambda x: [x[2],(x[3],1)])
rdd = rdd.reduceByKey(lambda a,b: (a[0]+b[0], a[1]+b[1])).map(lambda x: [x[0],x[1][0]/x[1][1]])
rdd = rdd.sortBy(lambda a: a[1])